In [1]:
import pandas as pd
import geopandas as geopd
import folium
from folium.plugins import FloatImage
from os import listdir
from os.path import isfile, join, splitext

In [8]:
path_geojson = '../../data/raw/Brazil/census_2010/digital_mesh/municipios.json'
path_data =  '../data/interim/Brazil/election_data/2018/polling_places/colected//concatenated_rooftop_tse.csv'
path_data_all =  '../data/interim/Brazil/election_data/2018/polling_places/colected//concatenated_all_precisions.csv'
path_city_names_to_correct = '../data/raw/Brazil/censu_2010/digital_mesh/names_to_correct_Censo_2010_Eleicoes_2018.csv'
path_state_codes = '../../data/raw/Brazil/censu_2010/digital_mesh/codigo_uf_ibge.csv'


#data_only_r_tse = pd.read_csv(path_data)
#data_all = pd.read_csv(path_data_all)
#city_names_to_correct = pd.read_csv(path_city_names_to_correct)
digital_mesh = geopd.read_file(path_geojson)
#states_codes = pd.read_csv(path_state_codes)

#data_all = data_all.drop_duplicates(subset='id_unico').reset_index()
#data_only_r_tse = data_only_r_tse.drop_duplicates(subset='id_unico').reset_index()
digital_mesh.dtypes


NM_MUNICIP      object
CD_GEOCMU       object
geometry      geometry
dtype: object

In [162]:
def get_state_codes_from_city_codes(city_codes):
    list_cod_uf = []
    for code in city_codes: 
        cod_uf = int(str(code[:2]))
        list_cod_uf.append(cod_uf)
    return list_cod_uf

digital_mesh['CD_GEOCUF'] = get_state_codes_from_city_codes(digital_mesh['CD_GEOCMU'])
digital_mesh = pd.merge(left=digital_mesh, right = states_codes, on='CD_GEOCUF', how='left')
digital_mesh['idx_1'] = digital_mesh['NM_MUNICIP']
digital_mesh['idx_2'] = digital_mesh['SIGLA']

digital_mesh.set_index(['idx_1','idx_2'], inplace=True)
city_names_to_correct.set_index(['census_name','uf'], inplace=True)

digital_mesh.update(city_names_to_correct)

In [163]:
locals_only_r_tse = data_only_r_tse.LOCALIDADE_LOCAL_VOTACAO.value_counts()
locals_all = data_all.LOCALIDADE_LOCAL_VOTACAO.value_counts()

locals_all['ABDON BATISTA']

7

In [164]:
perc_r_tse = (locals_only_r_tse*100)/locals_all
perc_r_tse.fillna(0, inplace=True)
perc_r_tse = pd.DataFrame(perc_r_tse)
perc_r_tse['NM_MUNICIP'] = perc_r_tse.index.values
perc_r_tse['TOTAL'] = locals_all
perc_r_tse['COLETADOS'] = locals_only_r_tse

In [165]:
merged = digital_mesh.merge(perc_r_tse, on='NM_MUNICIP', how='left')
merged

,NM_MUNICIP,CD_GEOCMU,geometry,CD_GEOCUF,UF,SIGLA,LOCALIDADE_LOCAL_VOTACAO,TOTAL,COLETADOS
0,ABADIA DE GOIÁS,5200050,POLYGON ((-49.38398960200001 -16.8318139509999...,52,Goiás,GO,100.000000,5.0,5.0
1,ABADIA DOS DOURADOS,3100104,POLYGON ((-47.42967244699997 -18.1654308179999...,31,Minas Gerais,MG,66.666667,6.0,4.0
2,ABADIÂNIA,5200100,POLYGON ((-48.61296496699999 -16.1428298059999...,52,Goiás,GO,100.000000,9.0,9.0
3,ABAETÉ,3100203,POLYGON ((-45.13929899999989 -19.2104779969999...,31,Minas Gerais,MG,41.666667,12.0,5.0
4,ABAETETUBA,1500107,POLYGON ((-48.53041071999996 -1.69932177399992...,15,Pará,PA,25.619835,121.0,31.0
5,ABAIARA,2300101,POLYGON ((-38.96954217199993 -7.42639766699998...,23,Ceará,CE,16.666667,12.0,2.0
6,ABAÍRA,2900108,POLYGON ((-41.60519754299997 -13.4248542309999...,29,Bahia,BA,0.000000,21.0,NaN
7,ABARÉ,2900207,POLYGON ((-39.35678600799995 -8.54745897399992...,29,Bahia,BA,18.181818,11.0,2.0
8,ABATIÁ,4100103,POLYGON ((-50.23476196899995 -23.3567263859999...,41,Paraná,PR,100.000000,3.0,3.0
9,ABDON BATISTA,4200051,POLYGON ((-50.96188439899996 -27.5548821359999...,42,Santa Catarina,SC,0.000000,7.0,NaN


In [166]:
merged.loc[merged['LOCALIDADE_LOCAL_VOTACAO'].isna(),:]
merged.fillna(0, inplace=True)

In [167]:
import branca.colormap as cm
statistics = merged['LOCALIDADE_LOCAL_VOTACAO']

index = pd.cut(statistics, bins = 7, retbins=True)[1].tolist()

linear = cm.LinearColormap(['#F9F9F3', '#bedbea', '#59a2cb', '#2166ac'],
                          vmin=merged['LOCALIDADE_LOCAL_VOTACAO'].min(), vmax=merged['LOCALIDADE_LOCAL_VOTACAO'].max())

step = cm.StepColormap(
    ['#2166ac','#59a2cb','#bedbea','#F9F9F3','#facab2', '#df745b', '#b2182c'],
    vmin=merged['LOCALIDADE_LOCAL_VOTACAO'].min(), vmax=merged['LOCALIDADE_LOCAL_VOTACAO'].max(),
    index=index,
    caption='Percentual Locais'
)

linear
step = linear

In [168]:
#geojson_style = {'fillColor': color_values, 'color': '#00000000'}

legend_name  = 'Percs'

m = folium.Map(location=[-14.2350, -51.9253], zoom_start=5, tiles='cartodbpositron')

# Add the color for the chloropleth:
folium.Choropleth(
 geo_data=path_geojson,
 name='Choropleth',
 data=merged,
 columns=['CD_GEOCMU', 'LOCALIDADE_LOCAL_VOTACAO'],
 key_on='feature.properties.CD_GEOCMU',
 fill_color= 'Paired',
 fill_opacity=0.7,
 line_opacity=0.2,
 legend_name= legend_name
)

folium.GeoJson(data=merged,
               name='Br Cities',
               tooltip=folium.GeoJsonTooltip(fields=['NM_MUNICIP', 'SIGLA', 'LOCALIDADE_LOCAL_VOTACAO', 'TOTAL', 'COLETADOS'],
                                             aliases=['<div style="background-color: lightyellow; color: black; padding: 3px; border: 2px solid black; border-radius: 3px;">'+item+'</div>' for item in ['City','State','Percentual', 'Total', 'Coletados']],
                                             labels=True,
                                             sticky=True),
               style_function=lambda city: {'fillColor': step(city['properties']['LOCALIDADE_LOCAL_VOTACAO']),
                                           'color': 'black',
                                           'fillOpacity': 0.7,
                                           'lineOpacity':0.1,
                                           'weight': 0.3
                                          }
              ).add_to(m)


m.add_child(step)

#FloatImage('https://i.ibb.co/pRQmCBR/legend-hotspots.png', bottom=75, left=80).add_to(m)

#folium.LayerControl().add_to(m)


 
# Save to html
m.save('../reports/figures/locais_votacao_coletados_TSE_ROOFTOP.html')
